In [1]:
#Definindo finais de semana
mes = "07"
lista_fds = ['06','07','13','14','20','21','27','28','31']

# Funções e Pacotes

In [2]:
# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#definindo quantidade de linhas e colunas visiveis
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# Manipulação de Strings
import unidecode # => unidecode(string)
import re # Regex

# Repositorio - Diretorio
import os
import shutil

In [3]:
#######################-FUNÇÕES DE CONTAGEM
def count_freq(data_frame, coluna):
    freq = df[coluna].value_counts()
    perc = df[coluna].value_counts(normalize = True)*100
    tabela = pd.DataFrame({"Freq":freq, "Perc":perc})
    return display(tabela)
        
def tabela_cont(linha, coluna):
    a = df.groupby([linha, coluna]).size().unstack()
    for i in a.columns:
        a[i].fillna(0, inplace = True)
        a[i] = a[i].apply(lambda x: int(x))
    return a
    print('='*100)

def tabela_cont_percent(linha, coluna, opcao):
    result = tabela_cont(linha, coluna)
    if opcao == 'Linha':
        return round(result.div(result.sum(axis=1), axis=0) * 100, 2)
    elif opcao == 'Coluna':
        return round(result.div(result.sum(), axis=1) * 100, 2)
    else:
        return 'Opção invalida'

In [4]:
# Funções de salvar arquivo
def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        len_antigo = len(dataframe)
        
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe = dataframe.drop_duplicates(subset=['chat'])
        
        print(len(df1), " += ", len_antigo, "=>", len(dataframe))
        
        dataframe.to_excel(nome_arquivo, index=False)
    
    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print(len(dataframe)) 

In [5]:
def read_file(file_path):
    # 1. Ler o arquivo txt
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    
    # 2. Remover todas as linhas que contêm "Mensagem apagada"
    lines = [line for line in lines if "Mensagem apagada" not in line]
    
    # 3. Corrigir as mensagens que estão em várias linhas para que fiquem em uma única linha
    corrected_lines = []
    current_message = ""
    
    for line in lines:
        # Se a linha começa com o padrão de data e hora, ela é uma nova mensagem
        if re.match(r"^\[\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}\]", line):
            if current_message:  # Se há uma mensagem anterior, adicione-a à lista
                corrected_lines.append(current_message)
            current_message = line.strip()  # Comece uma nova mensagem
        else:
            current_message += " " + line.strip()  # Adicione o conteúdo da linha atual à mensagem em progresso
    
    # Adiciona a última mensagem, se houver
    if current_message:
        corrected_lines.append(current_message)
    
    # Salvando arquivo txt
    with open('processed.txt', "w", encoding="utf-8") as file:
        for line in corrected_lines:
            file.write(line + "\n")
    
    # Lendo arquivo txt e transformando em dataframe cada mensagem
    with open('processed.txt', 'r', encoding='utf-8') as file:
        content = file.read().splitlines()
    
    return content

# =========================================

# Carregando base

In [6]:
df = pd.DataFrame({'chat': read_file('_chat.txt') })
#df['chat'] = df['chat'].apply(lambda x: x.replace('/05/2024, ','/05/2024 '))
df['controle'] = np.nan
df = df[['controle','chat']]
df_prob = pd.DataFrame(columns = ['controle','chat'])

print(len(df))
display(df.head())
display(df_prob)

660


,controle,chat
0,NaN,[01/07/2024 04:50:14] Gabriel de Aguiar Santan...
1,NaN,[01/07/2024 05:02:31] Henri Roger Watanabe Oli...
2,NaN,[01/07/2024 05:04:50] Mariana Gasparelli de So...
3,NaN,[01/07/2024 05:12:51] Maíra Leite Basile: 01/0...
4,NaN,[01/07/2024 05:15:58] Emerson Domingos da Cost...


,controle,chat


# Dia de envio da mensagem

In [8]:
dia = []
for i in range(0, len(df['chat'])):
    dia.append(df['chat'][i][1:11]) #pegando exatamente aonde parte da data esta
df['dia'] = dia
#print(sorted(df['dia'].unique()))
print("Quantidade de mensagens por dia:")
count_freq(df, 'dia')

Quantidade de mensagens por dia:


,Freq,Perc
dia,,
02/07/2024,238,36.060606
01/07/2024,223,33.787879
03/07/2024,199,30.151515


# Nome Anestesista

In [9]:
df['nome'] = np.nan
count = 0
for i in range(0, len(df['chat'])):
    loc = df['chat'][i].find(']')+1 #digito data
    loc2 = df['chat'][i][loc+1:].find(':')+1 #digito nome
    nome = df['chat'][i][loc+1:loc+loc2]
    nome = nome.upper().strip().replace("  ", " ")
    nome = unidecode.unidecode(nome)
    df['nome'][i] = nome
    count += 1
    #print(count, "->", unidecode.unidecode(chat[i][loc+1:loc+loc2].upper()))

print("Profissionais que mandaram mensagem:")
sorted(df['nome'].unique())

Profissionais que mandaram mensagem:


['ALESSANDRE LUIZ BRAGA JUNIOR',
 'ALEXANDRE AMARAL LUCAS',
 'ALEXANDRE BERNARDINO',
 'ALEXSANDER TAKASHI HAMADA',
 'ALINE FRANCHINI GOULART LIMA',
 'AMANDA KIYOKO SOUZA KOSSAKA',
 'ANA CLARA MOURAO BARRETO',
 'ANA JUNG DOCKO',
 'ANDRE AUGUSTO BELLINI',
 'ANDRE DA COSTA AGUIAR VENTURA',
 'ANDREIA LOUREIRO MORI',
 'ANNA BEATRIZ ARANHA TREVISAN',
 'ANTONIO ALVARO DIAS LIMA DA CUNHA NETO',
 'ARTHUR OLIVEIRA FACCHINI',
 'ARTHUR RODRIGUES TORRELIO',
 'AUGUSTO DE CASTRO SANTOS QUEIROZ',
 'BEATRIZ ANTIPOU DOS SANTOS',
 'BIANCA PRISCILA BUNIOTTI',
 'BRIAN SOSSAI PACHECO',
 'BRUCE YE MAN CHOW',
 'BRUNA DAIBERT DANESIN',
 'BRUNO FERRAZ DE CAMPOS BARONE',
 'BRUNO STORNIOLI TAGUCHI LEMOS',
 'CAIO TULIO DUARTE',
 'CAMILA CHIARANTANO HARRISON',
 'CARLOS AUGUSTO TREVISAN',
 'CARLOS EDUARDO GONDIM OLIVEIRA',
 'CAROLINE FIRMINO LEMES DA SILVA',
 'CLAUDIO MUELLER KAKUDA',
 'DANIEL MASUZAKI WONG',
 'DANIEL RODRIGUES DE OLIVEIRA',
 'DANIELLEN LINS LOURENCO',
 'DARLAN JUNIOR SCHNEIDER',
 'DEBORA ALVES AUDE

# Mensagens escrevidas e enviadas

In [10]:
mensagem = []
for i in range(0, len(df)):
    digito_data = df['chat'][i].find(']') #aonde acaba a data
    digito_nome = df['chat'][i][digito_data+1:].find(':')
    mensagem_escrevida = df['chat'][i][digito_data+digito_nome+3:]
    mensagem.append(mensagem_escrevida)
    
df['mensagem'] = mensagem
df['mensagem'] = df['mensagem'].apply(lambda x: x.upper() )
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('-',' - '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('(',' ( '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace(')',' ) '))

print('Mensagens Enviadas:')
df['mensagem'].sample(3)

Mensagens Enviadas:


629            03/07 MORUMBI LEMES SAÍDA  17:10
297           02/07 ANÁLIA ISAQUE ENTRADA 07:00
478    03/07 JABAQUARA  -  THALES ENTRADA 05:30
Name: mensagem, dtype: object

# =========================================

# Data do Platão

In [11]:
df['data'] = np.nan

for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar apenas registros com uma data apenas
        df['data'][i] = lista_datas[0]
    else: #restante com nenhuma ou mais de uma data
        df['controle'][i] = 'DATA_DO_PLANTÃO'

#salvando
df_prob = df[df['controle'] == 'DATA_DO_PLANTÃO']
print("data probs:", len(df_prob))
sample_prob('DATA_DO_PLANTÃO')

df = df[df['controle'] != 'DATA_DO_PLANTÃO']
print("datas ok:", len(df)) 

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

count_freq(df, 'data')
#print("Dias mensionados nas mensagens:")
#print(sorted(df['data'].unique()))

data probs: 4
datas ok: 656


,Freq,Perc
data,,
02/07,233,35.518293
01/07,228,34.756098
03/07,191,29.115854
1/07,2,0.304878
2/07,1,0.152439
2/7,1,0.152439


# Unidades

In [12]:
df['unidade'] = ''
df['Q_unidades'] = ''

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    
    #Unidades escrita com a bunda
    lista_avi = ["CENTRAL","AVICENA","AVICENNA","AVICCENNA"]
    for i_avi in lista_avi:
        mensagem = mensagem.replace(i_avi, 'AVICCENA')
        
    mensagem = mensagem.replace('Sta Isabel', 'SANTA')
    
    lista_unidades = re.findall(r'ALPHA|ANALIA|AVICCENA|ITAIM|JABA|MATERNIDADE|MORUMBI|OSASCO|VILLA|VILA|VNS|SANTA', mensagem)
    quantidade_unidades = len(lista_unidades)
    
    df['Q_unidades'][i] = quantidade_unidades
    
    if quantidade_unidades == 1: #capturando apenas com uma unidade
        df['unidade'][i] = lista_unidades[0].replace('  ',' ').upper().strip()
    elif quantidade_unidades > 1: #com mais de uma unidade
        df['unidade'][i] = lista_unidades
        df['controle'][i] = 'DESLOCADO'
    else: #sem nenhuma unidade
        df['controle'][i] = 'UNIDADES'
        
print("quantidades de unidades presentes nas mensagens:")
count_freq(df, 'Q_unidades')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'UNIDADES']])
print("unidades probs:", len(df_prob))
sample_prob('UNIDADES')

df = df[df['controle'] != 'UNIDADES']

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
#print("unidade ok:", len(df)) 
#print("unidade ok deslocado:", len(df[df['controle'] == 'DESLOCADO']))

quantidades de unidades presentes nas mensagens:


,Freq,Perc
Q_unidades,,
1,624,95.121951
2,26,3.963415
0,5,0.762195
3,1,0.152439


unidades probs: 9


# Deslocados

In [13]:
#preparando para reconhecer corretamente
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCADA','DESLOCADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCAMENTO','DESLOCADO'))

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DESLOCADO', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes > 0:
        df['controle'][i] = 'DESLOCADO'
        
count_freq(df, 'controle')
# OBS: Plantões de deslocado não são mais excluido

,Freq,Perc
controle,,
DESLOCADO,27,100.0


# (Retirada) Distancia e Saude

In [14]:
#RETIRANDO REGISTROS DE DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE OU DISPENSA
df['controle2'] = np.nan
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSADA','DISPENSADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSA ','DISPENSADO'))

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DISPENSADO|SAUDE|DISTANCIA|D[0-9]', mensagem)
    quantidade_observacoes = len(observacoes)

    if quantidade_observacoes >= 1:
        df['controle'][i] = 'DISP_SAUDE_DIST'
        if quantidade_observacoes == 1:
            df['controle2'][i] = observacoes[0]
        
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'DISP_SAUDE_DIST']])
print("unidades probs:", len(df_prob))
sample_prob('DISP_SAUDE_DIST')

df = df[df['controle'] != 'DISP_SAUDE_DIST']
print("unidade ok:", len(df)) 

print(df_prob['mensagem'][df_prob['controle'] == 'DISP_SAUDE_DIST'].sample(2))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,26,60.465116
DISP_SAUDE_DIST,17,39.534884


unidades probs: 26
unidade ok: 634
632       03/07 VNS BARONE ENTRADA 07:00  ( DISTÂNCIA ) 
351    02/07 MATERNIDADE MOTA ENTRADA 15:00  ( ASSUMO...
Name: mensagem, dtype: object


In [15]:
for i in range(0, len(df)):
    mensagem = df['mensagem'][i].upper()
    regex_D = re.findall('D[0-9]|DISTANCIA', mensagem)
    
    if len(regex_D) == 0:
        continue
    else:
        df['controle'][i] == 'DISTANCIAS'

count_freq(df, 'controle')

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'DISTANCIAS']])
print("probs:", len(df_prob))
sample_prob('DISTANCIAS')

df = df[df['controle'] != 'DISTANCIAS']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,26,100.0


probs: 26
ok: 634


# Extras

In [16]:
df["extra"] = np.nan

for i in range(0, len(df)):
    if 'EXTRA' in df['mensagem'][i].upper(): #se conter a palavra "extra"
        df['extra'][i] = 'extra' #marque na coluna extra como extra
        
print("Quantidade de registros Extra - Voluntario:")
count_freq(df, 'extra')      

Quantidade de registros Extra - Voluntario:


,Freq,Perc
extra,,


# Tipo entrada e saida

In [17]:
#Indentificando se é um registro de entrada ou saida
df['tipo'] = np.nan

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    entradas_saidas = re.findall(r'ENTRADA|SAIDA', mensagem)
    quantidade_entradas_saidas = len(entradas_saidas)
    
    if quantidade_entradas_saidas != 1: #capturando apenas com uma informação escrita
        df['controle'][i] = 'SEM_ENTRADA_OU_SAIDA'
    else: #restante com mais de uma informação
        df['tipo'][i] = entradas_saidas[0]
        
count_freq(df, 'tipo')
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'SEM_ENTRADA_OU_SAIDA']])
print("unidades probs:", len(df_prob))
sample_prob('SEM_ENTRADA_OU_SAIDA')

df = df[df['controle'] != 'SEM_ENTRADA_OU_SAIDA']
print("unidade ok:", len(df)) 

#print(df_prob['mensagem'][df_prob['controle'] == 'SEM_ENTRADA_OU_SAIDA'].sample(1))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

,Freq,Perc
tipo,,
ENTRADA,349,55.134281
SAIDA,284,44.865719


,Freq,Perc
controle,,
DESLOCADO,25,96.153846
SEM_ENTRADA_OU_SAIDA,1,3.846154


unidades probs: 27
unidade ok: 633
ok: 633


# -----------------------------------------

# Data

In [18]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = np.nan

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio

In [19]:
#arrumando data d/mm (6/10, não 06/10)
for i in range(0, len(df)):
    if df['data'][i][1] == '/':
        df['data'][i] = '0'+df['data'][i]
df['data'].unique()

array(['01/07', '02/07', '02/7', '03/07'], dtype=object)

In [20]:
#somando informações restantes na data: dd/MM/yyyy
for i in range(0, len(df)):
    df['data'][i] = df['data'][i][0:2] + "/"+ mes +"/2024" 
    
count_freq(df, 'data')

,Freq,Perc
data,,
01/07/2024,225,35.545024
02/07/2024,225,35.545024
03/07/2024,183,28.909953


In [21]:
df['sem_ou_fds'] = 'semana'

for i in lista_fds:
    datinha = i+"/"+mes+"/2024"
    df['sem_ou_fds'][df['data'] == datinha] = 'fds'

count_freq(df, 'sem_ou_fds')

,Freq,Perc
sem_ou_fds,,
semana,633,100.0


In [22]:
print(df.groupby(['data', 'sem_ou_fds']).size().unstack())

sem_ou_fds  semana
data              
01/07/2024     225
02/07/2024     225
03/07/2024     183


In [23]:
#df['data'].iloc[180] = '26/09/2022'

# Hora

In [24]:
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

In [25]:
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)
    
    if quantidade_horas == 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle'][i] = 'HORA'
        
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'HORA']])
print("probs:", len(df_prob))
sample_prob('HORA')

df = df[df['controle'] != 'HORA']
print("ok:", len(df)) 

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,25,80.645161
HORA,6,19.354839


probs: 33
ok: 627


In [26]:
#adicionando 0 no primeiro algorismo dos valores h:mm
for i in range(0, len(df)):
    if df['hora'][i][1] == ":":
        df['hora'][i] = "0"+df['hora'][i]
        
df['hora'].unique()

array(['04:45', '05:00', '05:15', '05:10', '05:20', '06:00', '05:45',
       '05:30', '06:25', '07:00', '06:30', '05:40', '07:09', '07:40',
       '08:00', '09:00', '09:10', '09:35', '09:50', '11:00', '10:30',
       '09:30', '10:50', '10:40', '10:57', '11:10', '11:45', '12:40',
       '12:20', '12:00', '13:00', '13:15', '13:35', '13:40', '14:45',
       '15:00', '15:07', '16:00', '15:50', '16:25', '15:45', '17:00',
       '16:45', '16:55', '17:05', '17:10', '17:15', '17:19', '17:20',
       '17:25', '17:30', '17:35', '17:45', '17:50', '18:00', '18:10',
       '18:15', '18:37', '18:40', '18:45', '18:50', '18:55', '19:10',
       '19:25', '19:30', '19:35', '19:55', '20:05', '20:10', '17:40',
       '20:25', '20:20', '20:35', '20:40', '20:45', '21:10', '21:20',
       '21:40', '21:55', '22:15', '23:10', '04:15', '04:40', '04:50',
       '05:35', '04:10', '06:15', '05:50', '22:35', '06:10', '09:15',
       '05:25', '10:35', '11:15', '11:38', '12:10', '13:20', '14:15',
       '14:43', '14:

# Hora final

In [ ]:
#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

In [ ]:
df['hora'].unique()

In [ ]:
#criando uma coluna datetime para fazer calculos de horas e minutos
df['hora2'] = df['hora'].apply(lambda x: datetime.strptime(x, '%H:%M'))

#verificando se esta tudo certo
print(df['hora2'].dt.year.unique())
print(df['hora2'].dt.month.unique())
print(df['hora2'].dt.day.unique())
print(df['hora2'].dt.hour.unique())

for i in range(0, len(df)):
    minuto = int(df['hora'][i][3:])
    
    if minuto != 0 and minuto != 30:
        if minuto < 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (0) subtraindo quantos minutos tem entre o momento e HH:00
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-minuto))[11:-3]
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (30) acrecentando quantos minutos faltam para HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(30-minuto)))[11:-3]
            else:
                df['controle'][i] = 'Hora_final'
                
        elif minuto > 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (30) subtraindo quando minutos tem entre o momento e HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-(minuto-30)))[11:-3]   
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (60) acrecentando quantos minutos faltam para HH:60
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(60-minuto)))[11:-3]            
            else:
                df['controle'][i] = 'Hora_final'
        else:
            df['controle'][i] = 'Hora_final'
            
    elif minuto == 0 or minuto == 30:
        df['hora'][i] = str(df['hora2'][i])[11:-3] 
        
    else:
        df['controle'][i] = 'Hora_final'

df['hora'].unique()

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'Hora_final']])
print("probs:", len(df_prob))
sample_prob('Hora_final')

df = df[df['controle'] != 'Hora_final']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

# Setor

In [ ]:
#reconhendo em qual periodo do dia aconteceu o plantão
df['ENTRADA'] = ''
df['SAIDA'] = ''

df['PERIODO'] = ''

for i in range(0, len(df)):
    minuto = int(df['hora'][i][:2])
    
    if df['tipo'][i] == 'ENTRADA':
        df['ENTRADA'][i] = df['hora'][i]
        if minuto < 7: #periodo 6h
            df['PERIODO'][i] = '6h'
            df['SAIDA'][i] = '07:00'
        elif minuto >= 7 and minuto < 13: #periodo manha
            df['PERIODO'][i] = 'manha'
            df['SAIDA'][i] = '13:00'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            df['PERIODO'][i] = 'tarde'
            df['SAIDA'][i] = '19:00'
        else:
            df['controle'][i] = 'HORARIOS'
    elif df['tipo'][i] == 'SAIDA':
        df['SAIDA'][i] = df['hora'][i]
        if minuto >= 7 and minuto < 13: #periodo manha
            df['ENTRADA'][i] = '07:00'
            df['PERIODO'][i] = 'manha'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            if df['hora'][i] == '13:00':
                df['SAIDA'][i] = '13:00'
                df['ENTRADA'][i] = '07:00'
                df['PERIODO'][i] = 'manha'
            else:
                df['ENTRADA'][i] = '13:00'
                df['PERIODO'][i] = 'tarde'
        elif minuto >= 19: #periodo noturno
            df['ENTRADA'][i] = '19:00'
            df['PERIODO'][i] = 'noturno saida'
        else:
            df['controle'][i] = 'HORARIOS'
    else:
        df['controle'][i] = 'HORARIOS'
        
count_freq(df, 'PERIODO')
count_freq(df, 'controle')

df['periodo'] = np.nan
df['periodo'][df['PERIODO'] == '6h'] = 1
df['periodo'][df['PERIODO'] == 'manha'] = 2
df['periodo'][df['PERIODO'] == 'tarde'] = 3
df['periodo'][df['PERIODO'] == 'noturno saida'] = 4

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'HORARIOS']])
print("probs:", len(df_prob))
sample_prob('HORARIOS')

df = df[df['controle'] != 'HORARIOS']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

# Duração do Plantão

In [ ]:
df['DURAÇÃO'] = ''

for i in range(0, len(df)):
    diferenca_tempo = datetime.strptime(df['SAIDA'][i], '%H:%M') - datetime.strptime(df['ENTRADA'][i], '%H:%M')
    
    #diferenca_tempo vem em formato estranho, tem-se que transformar em segundos antes
    diferenca_tempo_s = diferenca_tempo.seconds
    horas = '0' + str(int((diferenca_tempo_s/60)/60)) #capturando horas
    minutos = str(int((diferenca_tempo_s/60)%60))  #capturando minutos
    
    if len(minutos) == 1:
        minutos = '0' + minutos
    
    df['DURAÇÃO'][i] = horas + ':' + minutos
    
df['DURAÇÃO'][df['DURAÇÃO'] == '00:00'] = '06:00'    

df['DURAÇÃO'].unique()

# *****************************************

df['setor'] = ''

for i in range(0, len(df)):
    if df['sem_ou_fds'][i] == 'semana':
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'SEMANA - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'SEMANA - MANHA - C.CIRURGICO'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'SEMANA - TARDE - C.CIRURGICO'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'SEMANA - NOTURNO - SAIDA'
        else:
            df['setor'][i] = 'VERIFICAR'

    elif df['sem_ou_fds'][i] == 'fds': 
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'FDS - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'FDS - C.CIRURGICO ROTINA'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'FDS - SAIDA'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'FDS - PLANTAO'
        else:
            df['setor'][i] = 'VERIFICAR'  

    else:
        df['setor'][i] = 'VERIFICAR'
        
count_freq(df, 'setor')

# *****************************************

# Pareamento

In [ ]:
print(len(df))
print(len(df_prob))

627
33


In [ ]:
df.sample(2)

,controle,chat,dia,nome,mensagem,data,unidade,Q_unidades,controle2,extra,tipo,hora,sem_ou_fds
621,NaN,[03/07/2024 18:06:35] Juliana Akemi Funabashi:...,03/07/2024,JULIANA AKEMI FUNABASHI,03/07 ANÁLIA - AKEMI SAÍDA 17:50,03/07/2024,ANALIA,1,NaN,NaN,SAIDA,17:50,semana
447,DESLOCADO,[03/07/2024 05:10:04] Matheus Viana Lemo: 03/0...,03/07/2024,MATHEUS VIANA LEMO,03/07 MORUMBI VIANA ENTRADA 05:00 ( DESLOCADO...,03/07/2024,"[MORUMBI, JABA]",2,NaN,NaN,ENTRADA,05:00,semana


In [ ]:
nome = 'FABIO ANTONIO DE SOUZA OLIVEIRA'
data = '01/07/2024'

if data == '':
    display(df[df['nome'] == nome])
else:
    display(df[(df['nome'] == nome) & (df['data'] == data)])

# Procurar se o registro foi mandado com o erro:
df_prob_filter = df_prob[(df_prob['nome'] == nome) & (df_prob['data'] == data)]

if len(df_prob_filter) > 0:
    display(df_prob_filter)

,controle,chat,dia,nome,mensagem,data,unidade,Q_unidades,controle2,extra,tipo,hora,sem_ou_fds
139,NaN,[01/07/2024 17:19:53] Fábio Antônio de Souza O...,01/07/2024,FABIO ANTONIO DE SOUZA OLIVEIRA,01/07 ITAIM FABIO SAÍDA 17:19HS,01/07/2024,ITAIM,1,NaN,NaN,SAIDA,17:19,semana


,controle,chat,dia,nome,mensagem,data,unidade,Q_unidades,controle2,extra,tipo,hora,sem_ou_fds
27,HORA,[01/07/2024 05:27:10] Fábio Antônio de Souza O...,01/07/2024,FABIO ANTONIO DE SOUZA OLIVEIRA,01/07 ITAIM FABIO ENTRADA 5HS,01/07/2024,ITAIM,1,NaN,NaN,ENTRADA,,semana


In [ ]:
df_prob_pareado = pd.DataFrame(columns=df.columns)
df_prob_pareado

,controle,chat,dia,nome,mensagem,data,unidade,Q_unidades,controle2,extra,tipo,hora,sem_ou_fds


In [ ]:
count_ok = 0
count_erro = 0

for nome in sorted(df['nome'].unique()):
    df_filter = df[df['nome'] == nome]
    for data in sorted(df_filter['data'].unique()):
        df_filter = df[(df['nome'] == nome) & (df['data'] == data)]

        if len(df_filter) >= 2:
            qtd_entrada = len(df_filter[df_filter['tipo'] == 'ENTRADA'])
            qtd_saida = len(df_filter[df_filter['tipo'] == 'SAIDA'])

            if qtd_entrada == qtd_saida:
                print(nome, '|', data)
                count_ok += 1
        else:
            # Procurar se o registro foi mandado com o erro:
            df_prob_filter = df_prob[(df_prob['nome'] == nome) & (df_prob['data'] == data)]

            # ERRO PARCIAL
            if len(df_prob_filter) > 0: 
                print(nome, '|', data, '|', 'ERRO**********************************')
                
                # juntando os dois registros pareados e removendo do ok
                df_prob_pareado = pd.concat([df_prob_pareado, df_prob_filter, df_filter])
                df = df.drop(df_filter.index, axis=0).reset_index(drop=True)
                df_prob = df_prob.drop(df_prob_filter.index, axis=0).reset_index(drop=True)
                
            # ERRO DE PERDA TOTAL
            else: 
                print(nome, '|', data, '|', 'ERRO')

                # juntando aos probs e removendo do ok
                df_prob = pd.concat([df_prob, df_filter])
                df = df.drop(df_filter.index, axis=0).reset_index(drop=True)

            count_erro += 1
    print('='*50)

ALESSANDRE LUIZ BRAGA JUNIOR | 03/07/2024 | ERRO
ALEXANDRE AMARAL LUCAS | 02/07/2024
ALEXANDRE AMARAL LUCAS | 03/07/2024 | ERRO
ALEXANDRE BERNARDINO | 02/07/2024 | ERRO
ALEXANDRE BERNARDINO | 03/07/2024 | ERRO
ALEXSANDER TAKASHI HAMADA | 01/07/2024
ALEXSANDER TAKASHI HAMADA | 02/07/2024
ALINE FRANCHINI GOULART LIMA | 02/07/2024
AMANDA KIYOKO SOUZA KOSSAKA | 02/07/2024
AMANDA KIYOKO SOUZA KOSSAKA | 03/07/2024
ANA CLARA MOURAO BARRETO | 02/07/2024 | ERRO
ANA CLARA MOURAO BARRETO | 03/07/2024
ANA JUNG DOCKO | 01/07/2024
ANA JUNG DOCKO | 03/07/2024
ANDRE AUGUSTO BELLINI | 02/07/2024
ANDRE AUGUSTO BELLINI | 03/07/2024 | ERRO
ANDRE DA COSTA AGUIAR VENTURA | 02/07/2024
ANDRE DA COSTA AGUIAR VENTURA | 03/07/2024
ANDREIA LOUREIRO MORI | 01/07/2024
ANDREIA LOUREIRO MORI | 03/07/2024
ANNA BEATRIZ ARANHA TREVISAN | 02/07/2024
ANNA BEATRIZ ARANHA TREVISAN | 03/07/2024 | ERRO
ANTONIO ALVARO DIAS LIMA DA CUNHA NETO | 01/07/2024
ANTONIO ALVARO DIAS LIMA DA CUNHA NETO | 02/07/2024
ARTHUR OLIVEIRA FACCH

In [ ]:
print(count_ok)
print(count_erro)

# Checagem Unidades:

In [ ]:
# Cria uma nova Série sem as listas
serie_sem_listas = pd.Series([x for x in df['unidade'] if not isinstance(x, list)])

# Imprime os valores únicos da nova Série
print(serie_sem_listas.unique())

In [ ]:
# Cria uma lista com apenas os valores que são listas
listas = [x for x in df['unidade'] if isinstance(x, list)]

# Converte as listas em tuplas e adiciona elas a um conjunto para obter apenas as listas distintas
listas_distintas = set(tuple(lista) for lista in listas)

# Imprime as listas distintas
for lista in listas_distintas:
    print(list(lista))


In [ ]:
df['setor'] = df['setor'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))

#problemas com santa isabel 
#df['setor'][(df['unidade'] == "SANTA ISABEL") & (df['setor'] == "FDS - SAIDA")] = "FDS - NOTURNO - SAIDA"
df['setor'][(df['unidade'] == "SANTA ISABEL") & (df['setor'] == "FDS - C.CIRURGICO ROTINA")] = "FDS - MANHA - C.CIRURGICO"

In [ ]:
df['nome'] = df['nome'].apply(lambda x: x.replace('  ',' '))

#df['UNIDADE'] = df['UNIDADE'].apply(lambda x: x.replace('JABAQUARAQUARA','JABAQUARA'))
#df['UNIDADE'].unique()

# Ultimos ajustes:

In [ ]:
df = df[['chat', 'dia', 'nome', 'mensagem', 'controle', 'data', 'unidade',
       'extra', 'tipo', 'ENTRADA', 'SAIDA', 'DURAÇÃO', 'PERIODO','periodo',
         'setor', 'sem_ou_fds']]

# =========================================

# Deslocados:

In [ ]:
df['controle'][(df['PERIODO'] == '6h') & (df['controle'] == 'DESLOCADO')] = 'DESLOCADO 6H'
count_freq(df, 'controle')

In [ ]:
df['unidade2'] = df['unidade']

for i in range(0, len(df)):
    if isinstance(df.loc[i, 'unidade'], list):
        df['unidade'][i] = df['unidade'][i][0]

df['unidade'][df['unidade'] == 'VILA'] = 'VILLA'
df['unidade'][df['unidade'] == 'JABA'] = 'JABAQUARA'
df['unidade'][df['unidade'] == 'SANTA'] = 'SANTA ISABEL'

sorted(df['unidade'].unique())
count_freq(df, 'unidade')

# (1) Salvando tudo: gabs & giov

In [ ]:
salvando_nome_arquivo('df_gabs.xlsx', df)
print('.')
salvando_nome_arquivo('df_giov.xlsx', df_prob)

# =========================================

# Dispensados:

In [ ]:
df = pd.read_excel('df_giov.xlsx')[['chat','dia','nome','mensagem']]
df = df.drop_duplicates(subset=['chat'])
df = df.sort_values(['nome', 'dia'])
print(len(df))
df.sample(3)

In [ ]:
#RETIRANDO REGISTROS DE DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE OU DISPENSA

df['controle'] = np.nan

#preparando para reconhecer corretamente

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DISPENSADO|SAUDE|DISPENSADA', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes == 0:
        df['controle'][i] = 'PROB'
        
df['controle'].value_counts() 

#salvando
df_prob = df[df['controle'] == 'PROB']
df_prob.reset_index(drop=True, inplace=True)
print("unidades probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("unidade ok:", len(df)) 

In [ ]:
#DATA DO PLANTÃO
df['controle'] = ''
df['data'] = ''

for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar apenas registros com uma data apenas
        df['data'][i] = lista_datas[0]
        df['controle'][i] = 'NORM'
    else: #restante com nenhuma ou mais de uma data
        df['controle'][i] = 'PROB'

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] != 'NORM']])
df_prob.reset_index(drop=True, inplace=True)
print("data probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("datas ok:", len(df)) 

df['data'] = df['data']+"/2024"
sorted(df['data'].unique())

In [ ]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = ''

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio
    
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

#Retirando aqueles que contem horas no registro da mensagem
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)
    
    if quantidade_horas >= 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
        df['controle'][i] = 'PROB'
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle'][i] = 'OK'
        
print(df['controle'].value_counts())

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] != 'OK']])
df_prob.reset_index(drop=True, inplace=True)
print("probs:", len(df_prob))

df = df[df['controle'] == 'OK']
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

In [ ]:
print(len(df_prob))
print(len(df))

# (2) Salvando tudo: giov & dispensados

In [ ]:
df = df.drop_duplicates(subset=['chat'])

os.remove("df_giov.xlsx")
salvando_nome_arquivo('df_giov.xlsx', df_prob)
print('.')
salvando_nome_arquivo('df_dispensados.xlsx', df)

# ===========================================

# Ultima Contagem

In [ ]:
print(len(pd.DataFrame({'chat': read_file('_chat.txt') })))
print(len(pd.read_excel('df_dispensados.xlsx')))
print(len(pd.read_excel('df_gabs.xlsx')))
print(len(pd.read_excel('df_giov.xlsx')))